# 성동구 (1095)
- 작성자 : 황다은
- 작성일자 : 2020년 10월 19일
- ste 부분이 조금 섞여있음.

In [8]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [2]:
df = pd.read_csv("201019one_shot.csv")
soup = BeautifulSoup(df['html'][0], 'html.parser')

### 2. 간단한 전처리 하기

In [27]:
rows = soup.select('div.itembox')[1].select('li.item')

### 3. 저장할 DataFrame 미리 만들기

In [46]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [47]:
for i in tqdm(range(len(rows))):
    title = rows[i].select('.titlebox')
    numtitle = title[0].select('p')[0].text
    number = re.findall('\d+', numtitle)[0]
    num = '1095' + '0' * (5- len(str(number))) + str(number)
    cnd = title[0].select('p')[1].text
    syy = ''; syd = ''; pth = ''; rsd = ''; atn = ''; dch = ''; adt = ''; age = ''; sex = '';
    
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
    patient = pd.concat([patient, new_pat])

    table = rows[i].select('div.covid_respon_table > table')
    if len(table) > 0:
        new_route = pd.read_html(str(table[0]))[0][1:]
        new_route = new_route.drop(new_route.columns[[0]], axis=1)
        new_route.columns = ['mob', 'frm', 'rgl', 'exd', 'ste']
        new_route['num'] = num
        new_route['ord'] = range(1, len(new_route)+1)
        new_route['dat'] = dat
        new_route['rgt'] = ''
        new_route['rgt'] = ''
        new_route['msk'] = ''
        new_route = new_route.loc[:,rou_col]

        route = pd.concat([route, new_route])
    
    else:
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
        route = pd.concat([route, new_route])


100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:01<00:00, 92.15it/s]


### 5. 결과 잘 나왔나 확인하기

In [49]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,109500129,2020-10-21,,,,2020-10-14,,,,,,
0,109500128,2020-10-21,,,,2020-10-08,,,,,,
0,109500127,2020-10-21,,,,2020-10-08,,,,,,
0,109500126,2020-10-21,,,,2020-10-08,,,,,,
0,109500125,2020-10-21,,,,2020-10-07,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
0,109500005,2020-10-21,,,,2020-03-20,,,,,,
0,109500004,2020-10-21,,,,2020-03-19,,,,,,
0,109500003,2020-10-21,,,,2020-02-29,,,,,,
0,109500002,2020-10-21,,,,2020-02-20,,,,,,


In [48]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
1,109500129,1,2020-10-21,-,,자택 → 타 지역,10.09(금)~10.11(일),,-,자차
2,109500129,2,2020-10-21,-,,타 지역 → 자택,10.12(월) 19:00~20:11,,-,자차
3,109500129,3,2020-10-21,-,,자택 ↔ 타 지역,10.12(월) 20:31 ~ 10.13(화) 06:57,,-,자차
4,109500129,4,2020-10-21,옥수동,,00아파트,10.13(화) 06:57~16:12,,자택 및 공용공간 소독완료 (안내문 부착),공동주택 (자택)
5,109500129,5,2020-10-21,-,,자택 → 병원,10.13(화) 16:12~16:16,,마스크착용,도보
...,...,...,...,...,...,...,...,...,...,...
0,109500005,1,2020-10-21,,,,,,,
0,109500004,1,2020-10-21,,,,,,,
0,109500003,1,2020-10-21,,,,,,,
0,109500002,1,2020-10-21,,,,,,,
